In [3]:
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 42.1 MB/s eta 0:00:00


In [4]:
import json
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

# Cargar datos de productos
with open('productos.json', 'r', encoding='utf-8') as f:
    productos = json.load(f)

# Cargar historial de compras
with open('historial_compras.json', 'r', encoding='utf-8') as f:
    historial_compras = json.load(f)

# Preparar el DataFrame de compras
compras_data = []
for compra in historial_compras:
    for producto in compra['productos']:
        compras_data.append({
            'id_cliente': compra['id_cliente'],
            'id_producto': producto['id_producto'],
            'cantidad': producto['cantidad_comprada']
        })

# Crear un DataFrame
df_compras = pd.DataFrame(compras_data)

# Crear matriz de interacción
matriz_interaccion = df_compras.pivot_table(index='id_cliente', columns='id_producto', values='cantidad', fill_value=0)

# Convertir a formato sparse
matriz_interaccion_sparse = csr_matrix(matriz_interaccion.values)

# Entrenar el modelo ALS
modelo = AlternatingLeastSquares(factors=10, regularization=0.1, iterations=50)
modelo.fit(matriz_interaccion_sparse)

# Función para obtener recomendaciones basadas en múltiples artículos en el carrito
def recomendar_por_articulos(ids_articulos, num_recomendaciones=5):
    # Encontrar clientes que compraron los artículos en el carrito
    clientes_con_articulos = df_compras[df_compras['id_producto'].isin(ids_articulos)]['id_cliente'].unique()

    # Obtener los productos comprados por esos clientes
    productos_comprados = df_compras[df_compras['id_cliente'].isin(clientes_con_articulos)]

    # Contar las compras de otros productos
    conteo_productos = productos_comprados.groupby('id_producto').sum()['cantidad'].reset_index()
    conteo_productos = conteo_productos.sort_values(by='cantidad', ascending=False)

    # Obtener los productos recomendados, excluyendo los artículos actuales
    productos_recomendados = conteo_productos[~conteo_productos['id_producto'].isin(ids_articulos)].head(num_recomendaciones)

    # Obtener detalles de los productos recomendados
    recomendaciones = [(prod['id_producto'], next(p['nombre'] for p in productos if p['id'] == prod['id_producto'])) for idx, prod in productos_recomendados.iterrows()]

    return recomendaciones

# Ejemplo de uso
articulos_en_carrito = [
    {
        "id": 8,
        "nombre": "CACTUS MINI",
        "precio": 45.0,
        "descripcion": "Una pequeña muestra del reino de las cactáceas, poco riego, ¡mucho sol!",
        "categoria": "Cactus"
    },
    {
        "id": 30,
        "nombre": "SUCULENTA PERLA",
        "precio": 160.0,
        "categoria": "Suculentas",
        "descripcion": "Una suculenta con hojas en forma de perlas que añaden un toque exótico",
    },
]

# Extraer solo los IDs de los artículos en el carrito
ids_articulos_en_carrito = [articulo["id"] for articulo in articulos_en_carrito]

recomendaciones = recomendar_por_articulos(ids_articulos_en_carrito)
print(f"Recomendaciones basadas en los artículos en el carrito: {recomendaciones}")


/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/50 [00:00<?, ?it/s]

Recomendaciones basadas en los artículos en el carrito: [(4, 'ECHEVERIA'), (37, 'ABONO DE LOMBRIZ'), (5, 'ECHEVERIA GOLD'), (2, 'CROTÓ'), (25, 'SISTEMA DE RIEGO')]
